## ＲＮＮ教程
- ＲＮN原理简介
- Basic RNN
- LSTM
- CharRNN项目

## 案例讲解
通过一个案例讲解如何使用ＲＮＮ．项目目的是用ＬＳＴＭ学习句子生成．训练数据来自于莎士比亚的著作，以文本的形式存储．我们使用多层多步的Ｎ＊Ｎ模型来对这个问题建模．
比如，输入： how are you?对应的标签就是：ow are you? h．通过上一个字符预测下一个字符．字符构成单词，单词构成句子．
所以训练集样本就是一个个字符，对应标签也是字符，并且是每个训练样本对应的下一个字符．

## 代码流程
### 数据输入

```python
self.inputs = tf.placeholder(tf.int32, shape=(self.batch_size, self.num_steps),
                                         name='inputs')
self.targets = tf.placeholder(tf.int32, shape=(self.batch_size, self.num_steps),
                              name='labels')
self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')

if not self.use_embedding:
    self.rnn_inputs = tf.one_hot(self.inputs, self.n_classes)
else:
    embedding = tf.get_variable('embedding', [self.n_classes, self.embedding_size])
    self.rnn_inputs = tf.nn.embedding_lookup(embedding, self.inputs)
 
```
**Ｎｏｔｅ:注意这里的维度匹配问题**
- batch_size:表示一次输入多少个句子，每个句子相当于普通意义下的一个样本．实际上，在模型内部还是一个一个字符输入的．只是使用`tf.nn.rnn_cell．dynamic()`可以自动的把多个字符串起来．看起来的效果就是输入了一个句子
- num_steps:句子的长度，有多少个字符，或者叫做ＲＮＮ的展开长度
- rnn_inputs.shape:　（ｂａｔch_size， num_steps，　dimension of each char）．由于每个字符被表示成了一个ｏｎｅ-hot的向量，因此输入句子中每个字符是多维的
- 对于汉字的话我们会显式的embedding一下，因此显然每个汉子对应的也是个多维向量

> 为什么汉字不用one-hot编码？
我们要对输入进行特征表示，计算机只认识数字．你输入字符a，它并不知道什么意思？我们需要把它表示成一段数字，这段数字能够很好的刻画这个字符或者标识这个字符．这就叫做特征提取．对于英文字符，我们可以使用one-hot的形式．什么是one-hot呢？比如我们有猫，狗老鼠三个动物，那么ｏｎｅ-hot表示方法就是：猫＝１００，狗＝０１０，老鼠＝００１．即用每一位表示一个动物，是哪个动物，就激活哪个位置．对于英文字符，总共也就２６个英文字母，加上标点符号等等，顶多也就１００个对象．如果用ｏｎｅ-hot表示，每个对象的向量长度肯定是小于１００的．这个是可以接受的．
如果是汉字呢？你可能晕了，中国文化如此博大精深，那汉字还能少的了．姑且不说那些生僻字，仅仅是常用的汉字也有３０００多个．如果用one-hot表示，每个样本都是３０００多维的，这肯定不行啊．所以需要引入ｅｍｂｅｄding，你可以理解它赋予了每个汉字一个更加紧凑的表征．

### 模型搭建
- 利用`tf.nn.rnn_cell.BasicLSTMCell(size)`创建一个基本的ＬＳＴＭ模块
**Note:**这里的ｓｉｚｅ其实相当于ＭＬＰ中隐含层神经元的个数．这个ｓｉｚｅ容易和ｎｕｍ_step参数混淆．num_step是在一个时间维度上将下面函数定义的cell复制num_steps次．所以每个ｃｅｌｌ都是一样的，包括权重都是一样的．而且每个cell里面有size个神经元，对应每个cell是size维的输出．

```python
def create_cell(lstm_size, keep_prob):
    lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    drop = tf.nn.rnn_cell.DeviceWrapper(lstm, output_keep_prob=keep_prob)
    return drop
```

- 堆叠多层的cell

```python
cell = tf.nn.rnn_cell.MultiRNNCell(
    [create_cell(self.lstm_size, self.keep_prob) for _ in range(self.n_layers)])
self.init_state = cell.zero_state(self.batch_size,
                                  tf.float32)  # state is the hidden state
```

- 在时间维度上展开，构成一个句子

```python
self.lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cell, self.rnn_inputs,
                                                        initial_state=self.init_state)
```

- 输出．上面的输出的是多步隐藏层输出．每个ｓｔｅｐ是个多维的（本例１２８维），为了得到最后的输出，还需要经过全连接和softmax才能得到每个字符出现的概率（所以最后输出的维度是：(num_steps, num_chars))

```python
seq_output = tf.concat(self.lstm_outputs, 1)
x = tf.reshape(seq_output, [-1, self.lstm_size])  # row: bath_size * num_steps

with tf.variable_scope('output'):
    w = tf.Variable(tf.truncated_normal([self.lstm_size, self.n_classes], stddev=0.1))
    b = tf.Variable(tf.zeros(self.n_classes))
    self.logits = tf.matmul(x, w) + b
    self.preds = tf.nn.softmax(self.logits, name='prob_pred')
```

- 计算损失

```python
label = tf.one_hot(self.targets, self.n_classes)
y = tf.reshape(label, self.logits.get_shape())
loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=y)
self.loss = tf.reduce_mean(loss)
```

- 设置优化器

```python
train_vars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, train_vars), self.grid_clip)
optimizer = tf.train.AdamOptimizer(self.lr)
self.optimizer = optimizer.apply_gradients(zip(grads, train_vars))
```
